In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sqlite3
import pandas
import pandas.io.sql
import tqdm
from sklearn import *
import ml_metrics

/Users/riri/.virtualenvs/analysis3/lib/python3.4/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/Users/riri/.virtualenvs/analysis3/lib/python3.4/site-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [8]:
class CategoricalMeanEstimator:
    def __init__(self, col):
        self.col = col
        self.cls = None
        self.global_mean = None
    def fit(self, X, y):
        self.cls_mean = y.groupby(X[self.col]).mean().to_frame('estimate_mean')
        self.global_mean = y.mean()
        
        self.cls_median = y.groupby(X[self.col]).median().to_frame('estimate_median')
        self.global_median = y.median()

        return self
    
    def predict(self, X):
        x = X[self.col].to_frame('col')
        res_mean = pandas.merge(x, self.cls_mean, left_on='col', right_index=True, how='left')
        res_median = pandas.merge(x, self.cls_median, left_on='col', right_index=True, how='left')
        return (res_mean.estimate_mean.fillna(self.global_mean),
                res_median.estimate_median.fillna(self.global_median))

In [9]:
class MultiCategoricalMeanEstimator:
    def __init__(self, cols):
        self.cols = cols
        self.cls = None
        self.global_mean = None
    def fit(self, X, y):
        groups = [X[col] for col in self.cols]
        self.cls = y.groupby(groups).median().to_frame('estimate').reset_index()
        self.global_mean = y.median()
        return self
    
    def predict(self, X):
        x = X[self.cols]
        res = pandas.merge(
            x, self.cls, 
            left_on=self.cols, right_on=self.cols, 
            how='left')
        return res.fillna(self.global_mean).estimate

In [10]:
con = sqlite3.connect('/tmp/data.sqlite3')
total = 53364883
data = None
chunksize = int(5e6)
try:
    data_iter = pandas.read_sql('''
        SELECT week_num,
               sales_depo,
               sales_channel,
               route_id,
               client_id,
               product_id,
               adjusted_demand,
               rand
          FROM data 
         WHERE adjusted_demand is not null 
               AND week_num < 8''', con=con, chunksize=chunksize)
    for f in tqdm.tqdm(data_iter, total=1+total//chunksize):
        # This halves the memory use :(
        for col in f:
            if f[col].dtype == np.int64:
                f[col] = f[col].astype(np.int32)
        if data is None:
            data = f
        else:
            data = pandas.concat([data, f])
finally:
    con.close()

In [11]:
series = {'adjusted_demand': data.adjusted_demand}
admissible_cols = ['week_num', 'sales_depo', 'sales_channel', 'route_id', 'client_id', 'product_id']

estimators = {}
for col in tqdm.tqdm(admissible_cols):
    est = CategoricalMeanEstimator(col)
    est.fit(data, data.adjusted_demand)
    estimators[col] = est
    mean_est, med_est = est.predict(data)
    series[col + '_mean'] = mean_est
    series[col + '_median'] = med_est

'''
if False:
    for c1, c2 in tqdm.tqdm([(c1, c2) for c1 in admissible_cols for c2 in admissible_cols if c1 != c2]):
        est = MultiCategoricalMeanEstimator([c1, c2])
        est.fit(data, data.adjusted_demand)
        series_name = c1 + '_' + c2
        series[series_name] = est.predict(data)
        test_series[series_name] = est.predict(test_data)
        del est
'''
    
train_X = pandas.DataFrame(series)
train_X['rand'] = data.rand
train_X['week_num'] = data.week_num
train_X['adjusted_demand'] = data.adjusted_demand
del series, data

In [12]:
train_X.head()

,adjusted_demand,client_id_mean,client_id_median,product_id_mean,product_id_median,route_id_mean,route_id_median,sales_channel_mean,sales_channel_median,sales_depo_mean,sales_depo_median,week_num_mean,week_num_median,rand,week_num
0,23,8.566820,6.0,6.676974,5.0,18.730287,5.0,15.159744,5,15.904455,5.0,6.955922,3,0,3
1,3,5.181818,6.0,23.236511,10.0,18.730287,5.0,15.159744,5,15.904455,5.0,6.955922,3,0,3
2,5,4.336364,3.5,5.725000,4.0,18.730287,5.0,15.159744,5,15.904455,5.0,6.955922,3,0,3
3,5,4.263158,4.0,5.725000,4.0,18.730287,5.0,15.159744,5,15.904455,5.0,6.955922,3,0,3
4,10,8.250000,8.0,3.098225,2.0,21.278614,6.0,15.159744,5,15.904455,5.0,6.955922,3,0,3


In [ ]:
con = sqlite3.connect('/tmp/train_test_data.sqlite3')
try:
    # Set up the table
    pandas.io.sql.to_sql(train_X.head(1), 'train_data', con=con, if_exists='replace', index=False)
    iterr = iter(train_X.iterrows())
    next(iterr)
    collector = []
    for _, row in tqdm.tqdm(iterr, total=train_X.shape[0]):
        collector.append(row.values)
        if len(collector) > 100000:
            insert_term = ','.join('?' * row.shape[0])
            con.executemany('insert into train_data values (%s)' % insert_term, collector)
            collector = []
    if collector:
        insert_term = ','.join('?' * row.shape)
        con.executemany('insert into train_data values (%s)' % insert_term, collector)
    con.commit()
finally:
    con.close()

  8%|▊         | 4261634/53364883 [02:57<14:24, 56798.77it/s]

In [ ]:
con = sqlite3.connect('/tmp/data.sqlite3')
total = 20815581
test_data = None
chunksize = int(5e6)
try:
    data_iter = pandas.read_sql('''
        SELECT week_num,
               sales_depo,
               sales_channel,
               route_id,
               client_id,
               product_id,
               adjusted_demand,
               rand
          FROM data 
         WHERE adjusted_demand is not null 
               AND week_num >= 8''', con=con, chunksize=chunksize)
    for f in tqdm.tqdm(data_iter, total=1+total//chunksize):
        # This halves the memory use :(
        for col in f:
            if f[col].dtype == np.int64:
                f[col] = f[col].astype(np.int32)
        if test_data is None:
            test_data = f
        else:
            test_data = pandas.concat([test_data, f])
finally:
    con.close()

In [ ]:
test_series = {'adjusted_demand': test_data.adjusted_demand}
for col in tqdm.tqdm(admissible_cols):
    mean_est, median_est = estimators[col].predict(test_data)
    test_series[col + '_mean'] = mean_est
    test_series[col + '_median'] = median_est
    
test_X = pandas.DataFrame(test_series)
test_X['rand'] = test_data.rand
test_X['week_num'] = test_data.week_num
test_X['adjusted_demand'] = test_data.adjusted_demand

#del test_series

In [ ]:
con = sqlite3.connect('/tmp/train_test_data.sqlite3')
try:
    # Set up the table
    pandas.io.sql.to_sql(test_X.head(1), 'test_data', con=con, if_exists='replace', index=False)
    iterr = iter(test_X.iterrows())
    next(iterr)
    collector = []
    for _, row in tqdm.tqdm(iterr, total=test_X.shape[0]):
        collector.append(row.values)
        if len(collector) > 100000:
            insert_term = ','.join('?' * row.shape[0])
            con.executemany('insert into train_data values (%s)' % insert_term, collector)
            collector = []
    if collector:
        insert_term = ','.join('?' * row.shape)
        con.executemany('insert into train_data values (%s)' % insert_term, collector)
    con.commit()
finally:
    con.close()